# Exercise 2

In [1]:
# MAIN LIBRARIES to use
using Optim
using Statistics
using ForwardDiff
using Plots
using LinearAlgebra
using CSV
using DataFrames
using StatsFuns

We have the following model:

$$
\begin{gather*}
    y_t^{*}=\alpha+\rho y_{t-1}^{*}+\beta x_t + \epsilon_t \label{eq10}\tag{10} \\
    y_t = y_t^{*}+\upsilon_t
\end{gather*}
$$

where $\epsilon_{t}$ and $\upsilon_{t}$ are independent Gaussian white noise errors. Suppose that $y_{t}^{*}$ is not observed, and instead we observe $y_{t}$. We estimate the equation

$$
\begin{equation}
    y_t=\alpha+\rho y_{t-1}+\beta x_t + \nu_t \label{eq11}\tag{11}
\end{equation}
$$

which will yield a biased and inconsistent estimator, since the errors are not exogenous anymore $\mathbb{E}[x_t\nu_t]\neq 0$.

We're told to consider as instruments to deal with this endogeneity: $Z=[1\,\, x_t\,\, x_{t-1}\,\, x_{t-2}]$, lags of $x_t$ which are correlated with $y_{t-1}$ as long as $\beta\neq 0$. By assumption, $\mathbb{E}[x_{t-s}\nu_t]=0\quad\forall s\geq0$. The functions that we're given to define these lags:

In [3]:
function lag(x::Array{Float64,2},p::Int64)
	n,k = size(x)
	lagged_x = [ones(p,k); x[1:n-p,:]]
end

function lag(x::Array{Float64,1},p::Int64)
	n = size(x,1)
	lagged_x = [ones(p); x[1:n-p]]
end	 


function  lags(x::Array{Float64,2},p)
	n, k = size(x)
	lagged_x = zeros(eltype(x),n,p*k)
	for i = 1:p
		lagged_x[:,i*k-k+1:i*k] = lag(x,i)
	end
    return lagged_x
end	

function  lags(x::Array{Float64,1},p)
	n = size(x,1)
	lagged_x = zeros(eltype(x), n,p)
	for i = 1:p
		lagged_x[:,i] = lag(x,i)
	end
    return lagged_x
end

lags (generic function with 2 methods)

## (2.a) Orthogonality conditions
Before we had to deal with a particular type of M-Estimator, the ML. In this second exercise, we are treating the other: the GMM estimator. This latter, as opposed to the ML, does not require any knowledge about the distribution function of our data. Instead, its objective function is described as follows:

$$
\begin{equation}
     Q_n(\theta)=-\frac{1}{2}g_n(\theta)\,' \hat{W}\, g_n(\theta),\quad g_n(\theta)\equiv \frac{1}{n}\sum_{i=1}^{n}g(x_i;\theta) \label{eq12}\tag{12}
\end{equation}
$$

Enclosed in the function $g(x_i;\theta)$ are some orthogonality conditions that we will try to minimize. In fact, we have seen that endogenous regressors yield biased and inconsistent estimates of the parameters of our models - a linear model, in this case. We will call _instrument_ to the estimators that exogenize our endogenous regressors by exploting the variability of exogenous variables. Let's consider the standard linear model:

$$
\begin{equation}
     y = X\theta + \nu \label{eq13}\tag{13}
\end{equation}
$$

with $X$ an $n\times k$ matrix of data. In our case, $X=[1\,\, y_{t-1}\,\, x_t]$: a matrix of size $n\times 3$. The assumption that the instruments $Z$ are exogenous can be expressed as follows:

$$
\begin{gather*}
     \mathbb{E}[z_t \nu_t]=0 \\
     \mathbb{E}[z_tx_t']\quad \text{has rank k} \label{eq14}\tag{14} 
\end{gather*}
$$

These are the **moment conditions**. The $l$ instruments give us a set of $l$ moments,

$$
\begin{gather*}
     g_t(\theta)=Z_t'\nu_t=Z_t'(y_t-\alpha-\rho y_{t-1}-\beta x_t)\\
     g_t(\theta)=\begin{pmatrix}
                    (y_t-\alpha-\rho y_{t-1}-\beta x_t) \\
                    x_t(y_t-\alpha-\rho y_{t-1}-\beta x_t) \\
                    x_{t-1}(y_t-\alpha-\rho y_{t-1}-\beta x_t) \\
                    x_{t-2}(y_t-\alpha-\rho y_{t-1}-\beta x_t)
                    \end{pmatrix} \label{eq15}\tag{15} 
\end{gather*}
$$

where $g_t$ is $l\times 1$ ($4\times 1$ here). The exogeneity of the instruments means that there are $l$ moment conditions - orthogonality conditions, that will be satisfied at the true value of $\theta$, which is $\theta_0$:

$$
\begin{equation}
     \boxed{\mathbb{E}[g_t(\theta_0)]=\mathbb{E}[Z_t'(y_t-X_t\theta_0)]=0} \label{eq16}\tag{16} 
\end{equation}
$$

Each of the $l$ moment equations corresponds to a sample moment, and we write these $l$ sample moments as follows:

$$
\begin{gather*}
     \boxed{\bar{g}(\theta)=\frac{1}{n} \sum_{t=1}^{n}g_t(\theta)=
     \begin{pmatrix}
                    \frac{1}{n} \sum_{t=1}^{n}(y_t-\alpha-\rho y_{t-1}-\beta x_t) \\
                   \frac{1}{n} \sum_{t=1}^{n} x_t(y_t-\alpha-\rho y_{t-1}-\beta x_t) \\
                    \frac{1}{n} \sum_{t=1}^{n}x_{t-1}(y_t-\alpha-\rho y_{t-1}-\beta x_t) \\
                    \frac{1}{n} \sum_{t=1}^{n}x_{t-2}(y_t-\alpha-\rho y_{t-1}-\beta x_t)
                    \end{pmatrix}
                    =\frac{1}{n} \sum_{t=1}^{n}Z_t'(y_t-\alpha-\rho y_{t-1}-\beta x_t)=\frac{1}{n}Z'\nu} \label{eq17}\tag{17} 
\end{gather*}
$$

Now, the intuition behind GMM is to choose an estimator for $\theta$, $\hat{\theta}$ that sets **these $l=4$ sample moments** as close to zero as possible.

But, what if the  equation is overidentified, as it is here?

### Identification

If the equation is **overidentified, as in our case,**, so that $l > k$, then we have more equations than we do unknowns, and in general it will not be possible to find a $\hat{\theta}$ that will set  all $l$ sample  moment conditions to exactly zero. In this case, we take an $l\times l$ weighting matrix $W$ and use it to construct a quadratic form in the moment conditions.This gives us the GMM objective function:

$$
\begin{equation}
     Q_n(\theta)= n\bar{g}(\theta)'W\bar{g}(\theta) \label{eq18}\tag{18} 
\end{equation}
$$

Here, $\bar{g(\theta)}$ is $m_n(\theta)$ on our course notes. I only use $g$ not to confuse it with the $m$ in the M-Estimators.  A GMM estimator for $\theta$ is the $\hat{\theta}$ that minimizes $Q_n(\theta)$:

$$
\begin{equation}
     \frac{\partial Q_n(\theta)}{\partial \theta}=0 \label{eq19}\tag{19} 
\end{equation}
$$

## (2. b) Compute $\hat{\theta}$ using two step GMM.

The two step GMM estimator:

1. We will first set the weight matrix to some positive definite matrix: $W=I$. Obtain the GMM estimator that minimizes $Q_n(\theta)=\bar{g(\theta)}'W\bar{g(\theta)}$.
2. Based on this initial estimate $\hat{\theta}$, we compute its moment contributions $g_t(\hat{\theta})$. Compute an estimate of $\Omega_{\infty}$ based on the moment contributions, say $\hat{\Omega}^{-1}$. The exact way to do this will depend upon the assumptions of the model. Given the estimate, compute the efficient GMM estimator which minimizes: $Q_n(\theta)=\bar{g(\theta)}'\hat{\Omega}^{-1}\bar{g(\theta)}$

In [4]:
# ------------------- MAIN FUNCTIONS ------------------------- 

# what is the best moment to use???

# moment condition
function GIVmoments(theta, data)
    data = [data lags(data,2)]
    data = data[3:end,:] # get rid of missings
    n = size(data,1)
    y = data[:,1]
    ylag = data[:,2]
    x = data[:,3]
    xlag = data[:,6]
    xlag2 = data[:,9]
    X = [ones(n,1) ylag x]
    e = y - X*theta
    Z = [ones(n,1) x xlag xlag2]
    m = e.*Z
    return m
end

function gmm(moments, theta, data, weight)
    # average moments
    m = theta -> vec(mean(moments(theta,data),dims=1)) # 1Xg   
    # GMM objective function
    obj = theta -> ((m(theta))'weight*m(theta))
    # Minimization
    thetahat, objvalue, converged = fminunc(obj, theta)
    # Derivative of average moments
    D = (ForwardDiff.jacobian(m, vec(thetahat)))' 
    # moment contributions at estimate
    mc_thetahat = moments(thetahat,data)
    return thetahat, objvalue, D, mc_thetahat, converged
end

# Unconstrained minimization problem    
function fminunc(obj, x; tol = 1e-08)
    results = Optim.optimize(obj, x, LBFGS(), 
                            Optim.Options(
                            g_tol = tol,
                            x_tol=tol,
                            f_tol=tol))
    return results.minimizer, results.minimum, Optim.converged(results)
    #xopt, objvalue, flag = fmincon(obj, x, tol=tol)
    #return xopt, objvalue, flag
end

fminunc (generic function with 1 method)

### A brief description of what the provided code does

First we generate data for some particular values of the parameters: $\theta=[\alpha_0,\rho_0,\beta_0]=[0.0,0.9,1.0]$. The data will be generated using the lag functions defined above. In this part of the code, the main aim is to generate the dependent variable that **we do not observe**, which is $y_t^{*}=\alpha+\rho y_{t-1}^{*}+\beta x_t+\epsilon_t$. We are assuming that both the exogenous regressor and the error term are normally distributed. 

In [5]:
# ------------------- SET UP ------------------------- 

n = 1000
x = randn(n) # an exogenous regressor
e = randn(n) # the error term
ystar = zeros(n)
alpha_0 = 0.0
rho_0 = 0.9
beta_0 = 1.0
theta = [alpha_0 rho_0 beta_0]
# generate the unobserved dependent variable
for t = 2:n
  ystar[t] = theta[1] + theta[2]*ystar[t-1] + theta[3]*x[t] + e[t]
end

Once $y_t^{*}$ is defined, we construct the observed variable $y_t$, assuming once again that the error $\upsilon_t$ is normally distributed with a variance of $\sigma^2$.

Then, we construct a matrix called _data_ which will hold the values of $y_t$, $y_{t-1}$ and $x_t$ (first observation omitted, because of the lag).

In [6]:
# generate the observed dependent variable by adding measurement error
sig = 1
y = ystar + sig*randn(n);
ylag = lag(y,1);
data = [y ylag x];
data = data[2:end,:]; # drop first observation, missing due to lag

In [7]:
# ------------------- GMM TWO-STEP ESTIMATION ------------------------- 

theta_trial = [1.0, 0.5, 0.5]                     # Trial value of parameter estimators
moments = (theta,data) -> GIVmoments(theta,data)  # Generate the moments function to send as an argument for the gmm()

# -------------- FIRST ESTIMATION ---------------
thetahat1, junk, junk, ms, junk = gmm(moments, theta_trial, data, I(4));
W = inv(cov(ms)); 
# use  thetahat1 to re-estimate by defining a specific weighting matrix

# -------------- SECOND ESTIMATION --------------
thetahat, junk, junk, ms, junk = gmm(moments, thetahat1, data, W);

# Compare the estimators
df = DataFrame(Thetas = ["True value θ_0", "First estimation θ1_hat", "Second estimation θ2_hat"], Values = [theta, thetahat1, thetahat])

,Thetas,Values
,String,Array…
1,True value θ_0,[0.0 0.9 1.0]
2,First estimation θ1_hat,"[0.0125454, 0.947209, 1.00944]"
3,Second estimation θ2_hat,"[0.0113542, 0.948974, 1.00857]"


In this example, both the first estimated $\theta$ and the second round one are quite close, which means that weighing all the moments equally was satisfactory. Thus, for the sake of efficiency in this particular problem, we could stay with the first estimation.